# 📊 AI Observability & Evaluation Setup

This notebook sets up the TruLens AI Observability framework for evaluating the Intelligent Multi-Stage RAG Chatbot.

## Features
- **🔍 Comprehensive Tracing**: Track all RAG components (retrieval, generation, orchestration)
- **📈 Evaluation Metrics**: RAG Triad (Context Relevance, Groundedness, Answer Relevance)
- **👤 User Feedback**: Collect and analyze user satisfaction scores
- **🎯 Performance Monitoring**: Track application performance over time
- **📊 Visual Dashboards**: Snowsight-based evaluation dashboards

## Prerequisites
- Snowflake account with Cortex LLM Functions enabled
- ACCOUNTADMIN role or equivalent privileges
- TruLens packages installed (see requirements.txt)

## Required Privileges
- `SNOWFLAKE.CORTEX_USER` database role
- `SNOWFLAKE.AI_OBSERVABILITY_EVENTS_LOOKUP` application role
- `CREATE EXTERNAL AGENT` privilege on the schema


## 🛠️ Step 1: Environment Setup

### ⚠️ **Important**: Permission Grant Syntax

When running the optional permission grants, use the correct syntax:

**✅ Correct:**
```sql
GRANT APPLICATION ROLE SNOWFLAKE.AI_OBSERVABILITY_EVENTS_LOOKUP TO USER demo_user;
```

**❌ Incorrect (will cause syntax error):**
```sql
GRANT APPLICATION ROLE SNOWFLAKE.AI_OBSERVABILITY_EVENTS_LOOKUP TO demo_user;
```

**Key difference**: Application roles require the `USER` keyword, unlike database roles.


In [ ]:
# Import required libraries
import os
import json
import pandas as pd
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.cortex import Complete
from typing import List, Dict, Any

# TruLens imports
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.run import Run, RunConfig

# Enable TruLens OpenTelemetry tracing
os.environ["TRULENS_OTEL_TRACING"] = "1"

# Get active Snowflake session
session = get_active_session()
print("✅ Connected to Snowflake successfully")


## 🗄️ Step 2: Create Observability Database and Schema


In [ ]:
# Create observability database and schema
session.sql("""
CREATE DATABASE IF NOT EXISTS AI_OBSERVABILITY_DB
""").collect()

session.sql("""
USE DATABASE AI_OBSERVABILITY_DB
""").collect()

session.sql("""
CREATE SCHEMA IF NOT EXISTS EVALUATION_SCHEMA
""").collect()

session.sql("""
USE SCHEMA EVALUATION_SCHEMA
""").collect()

print("✅ Created AI_OBSERVABILITY_DB.EVALUATION_SCHEMA")
